In [1]:
import dash
from dash import dcc, html, Input, Output, callback, ctx
import plotly.express as px
import numpy as np
import pandas as pd
import datetime
import json
from datetime import datetime as dt
import pathlib
import plotly.graph_objects as go 
import plotly.io as pio
import dash_bootstrap_components as dbc

In [2]:
def preprocesamiento(df):
    # Llenar las categorias vacias con 'No Identificado'
    df["Admit Source"] = df["Admit Source"].fillna("Not Identified")
    # Date
    # Formateo "checkin Time"
    df["Check-In Time"] = df["Check-In Time"].apply(
        lambda x: dt.strptime(x, "%Y-%m-%d %I:%M:%S %p")
    )  # String -> Datetime

    # Insertar día de la semana y hora del "Checking time"
    df["Days of Wk"] = df["Check-In Hour"] = df["Check-In Time"]
    df["Days of Wk"] = df["Days of Wk"].apply(
        lambda x: dt.strftime(x, "%A")
    )  # Datetime -> weekday string

    df["Check-In Hour"] = df["Check-In Hour"].apply(
        lambda x: dt.strftime(x, "%I %p")
    )  # Datetime -> int(hour) + AM/PM

    return df



In [3]:
# Leer los datos y limpiarlos
df = pd.read_csv("../dash-clinical-analytics_est/data/clinical_analytics.csv")
# realizar preprocesamiento de fecha
df = preprocesamiento(df)

In [31]:
df.head()

,Admit Source,Admit Type,Appt Start Time,Care Score,Check-In Time,Clinic Name,Department,Diagnosis Primary,Discharge Datetime new,Encounter Number,Encounter Status,Number of Records,Wait Time Min,Days of Wk,Check-In Hour
0,Emergency Room,Emergency,2014-01-02 11:38:50 PM,2,2014-01-02 23:24:00,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,14,Thursday,11 PM
1,Emergency Room,Emergency,2014-01-02 11:44:18 PM,2,2014-01-02 23:24:00,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,20,Thursday,11 PM
2,Emergency Room,Emergency,2014-01-02 11:47:11 PM,2,2014-01-02 23:24:00,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,23,Thursday,11 PM
3,Emergency Room,Emergency,2014-01-08 10:38:04 PM,4,2014-01-08 21:00:00,Madison Center,Emergency,NaN,NaN,PK559C587,Cancelled,1,98,Wednesday,09 PM
4,Emergency Room,Emergency,2014-01-09 12:00:26 AM,4,2014-01-08 22:47:00,Madison Center,Emergency,NaN,NaN,48559C587,Cancelled,1,73,Wednesday,10 PM


In [4]:
clinicas = df["Clinic Name"].unique()
sources = df["Admit Source"].unique()
df_filtrado = df.groupby(["Clinic Name", "Days of Wk", "Check-In Hour"]).sum("Numberof Records").reset_index()

# Fase 1

En esta fase se debe realizar una planificación de cómo se construirá el tablero.
Para lo cual deben dar respuesta a los siguientes cuestionamientos:

**1. Como estructuraran el Dashboard?**

RT: El diagrama que responde a esta pregunta está en la carpeta anexos con el nombre de "Respuesta_fase_1.png"

**2. ¿Qué componentes y estilos piensan usar y porqué?**

RT: Se pretende utilizar componentes de html, dash y bootstrap embebidos en python como por ejemplo: Headers, tablas, parrafos, contenedores(divs), datepickersrange, Dropdown, graph, 

**3. Qué interacciones deben tener estos componentes.**

RT: En primera instancia deben ser interactivos con el usuario, mostrar la información correspondiente y variar de acuerdo a los filtros seleccionados.

In [61]:
df_new = df_filtrado[(df_filtrado["Days of Wk"] == 'Friday') & (df_filtrado["Check-In Hour"] == '03 AM')]
#df_new = df_new[df_new["Check-In Hour"] == '03 AM']
df_new.head()

,Clinic Name,Days of Wk,Check-In Hour,Care Score,Number of Records,Wait Time Min
4,Lakeview Center,Friday,03 AM,98,32,2178
172,Madison Center,Friday,03 AM,234,79,5472
340,Surgery Center,Friday,03 AM,63,23,1672


#Fase 2

En esta fase se procederá con la estructura y la personalización (estilos) del
dashboard. Recuerden que pueden usar estilos css, personalización a través de
html o el uso de Bootstrap

In [5]:
# Estructura App
external_stylesheets = [dbc.themes.CERULEAN]
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content":
                "width=device-width, initial-scale=1"}],
    external_stylesheets=external_stylesheets
)
app.title = "Clinical Analytics Dashboard"

fig_hm = go.Figure(go.Heatmap(
    z=df_filtrado["Number of Records"],
    x=df_filtrado["Check-In Hour"],
    y=df_filtrado["Days of Wk"],
    text=df_filtrado["Number of Records"],
    texttemplate="%{text}"
))

contenedor = dbc.Container([
    dbc.Row([
        html.Div([
            html.H1("Creación de Dashboard", style={'textAlign':'center'}),
            html.H1("Interactivo", style={'textAlign':'center'})
        ], style={'marginBottom': 25, 'marginTop': 25})
    ]),
    dbc.Row([
        dbc.Col([
            html.P("Selecciona la clinica:", className="text-center"),
        ], md= 1),
        dbc.Col([
            dcc.Dropdown(clinicas, id="dropdown_1", persistence= True),
        ], md= 5),
        dbc.Col([
            html.P("Selecciona en rango de fecha:", className="text-center"),
        ], md= 2),
        dbc.Col([
            dcc.DatePickerRange(id="date_1"),
        ], md= 4),
    ]),
    dbc.Row([
        dbc.Col([
            html.P("Selecciona Origen Paciente:", className="text-center"),
        ], md= 2),
        dbc.Col([
            dcc.Dropdown(options=[{'label': s, 'value': s} for s in sources], value=sources[:3], id="dropdown_2", multi=True)
        ], md= 9),
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H3("Mapa de calor", style={'textAlign':'center'}),
            dcc.Graph(id="grafico_hm", figure=fig_hm),
        ])
    ]),
    dbc.Row([
        html.H3("Tiempo de espera", style={'textAlign':'center'}),
        dcc.Graph(id='salida_click'),
    ]),
    dbc.Row([
        html.H3("Calificación", style={'textAlign':'center'}),
        dcc.Graph(id='salida_click_score')
    ])

])

@app.callback(
    Output("grafico_hm", "figure"),  
    [
        Input("dropdown_1", "value"),
        Input("date_1", "start_date"),
        Input("date_1", "end_date"),
        Input("dropdown_2", "value")
    ]
)
def update_graph(value, start_date, end_date, dropdown2):
    entra = 'N'
    triggered_id = ctx.triggered_id

    # Reviso los valores que llaman
    print(value)
    print(dropdown2)
    print(start_date, end_date)

    
    # Si alguno de los filtros cambia, se actualiza el gráfico
    if triggered_id in ["dropdown_1", "dropdown_2", "date_1"]:

        entra ='S'
        # Empiezo a filtrar por los valores que recibe la función
        df_grafico = df.copy()
        
        if value is not None: 
            df_grafico = df_grafico[df_grafico["Clinic Name"] == value]

        if dropdown2 is not None:
            df_grafico = df_grafico[df_grafico["Admit Source"].isin(dropdown2)]

        if start_date is not None and end_date is not None:
            df_grafico = df_grafico[(df_grafico['Appt Start Time'] >= start_date) & (df_grafico['Appt Start Time'] <= end_date)]

        df_agrupado = df_grafico.groupby(["Clinic Name", "Days of Wk", "Check-In Hour"]).sum("Numberof Records").reset_index()

        # Genero el heatmap con los datos filtrados
        fig_hm = go.Figure(go.Heatmap(
            z=df_agrupado["Number of Records"],
            x=df_agrupado["Check-In Hour"],
            y=df_agrupado["Days of Wk"],
            text=df_agrupado["Number of Records"],
            texttemplate="%{text}"
        ))

        # Retorno la figura actualizada
        return fig_hm
    
    if entra != 'S':
        fig_hm = go.Figure(go.Heatmap(
            z=df_filtrado["Number of Records"],
            x=df_filtrado["Check-In Hour"],
            y=df_filtrado["Days of Wk"],
            text=df_filtrado["Number of Records"],
            texttemplate="%{text}"
        ))
    # Si no hay cambios en los filtros, retorno el gráfico original 
    return fig_hm


@app.callback(
    Output("salida_click", "figure"), 
    [Input("grafico_hm", "clickData"),
     Input("dropdown_1", "value")]
)
def salida_click(clickData, clinic_name):
    triggered_id = ctx.triggered_id

    fig=px.box()

    if (triggered_id == "grafico_hm"):

        df_grafico_cajas = df.copy()

        valor_x = clickData["points"][0]["x"]
        valor_y = clickData["points"][0]["y"]
                
        df_grafico_cajas_2 = df_grafico_cajas[(df_grafico_cajas["Days of Wk"] == valor_y) & (df_grafico_cajas["Check-In Hour"] == valor_x)]

        if clinic_name is not None:
            df_grafico_cajas_2 = df_grafico_cajas_2[df_grafico_cajas_2["Clinic Name"] == clinic_name]
            
        print(df_grafico_cajas_2)

        fig = px.box(df_grafico_cajas_2, y="Wait Time Min", facet_col = 'Admit Type')
    
    return fig

@app.callback(
    Output("salida_click_score", "figure"), 
    [Input("grafico_hm", "clickData"),
     Input("dropdown_1", "value")]
)
def salida_click_sc(clickData, clinic_name):
    triggered_id = ctx.triggered_id

    fig=px.box()

    if (triggered_id == "grafico_hm"):

        df_grafico_cajas = df.copy()

        valor_x = clickData["points"][0]["x"]
        valor_y = clickData["points"][0]["y"]
                
        df_grafico_cajas_2 = df_grafico_cajas[(df_grafico_cajas["Days of Wk"] == valor_y) & (df_grafico_cajas["Check-In Hour"] == valor_x)]

        if clinic_name is not None:
            df_grafico_cajas_2 = df_grafico_cajas_2[df_grafico_cajas_2["Clinic Name"] == clinic_name]
            
        print(df_grafico_cajas_2)

        fig = px.box(df_grafico_cajas_2, y="Care Score", facet_col = 'Admit Type')
    
    return fig



app.layout = html.Div(contenedor)


if __name__ == "__main__":
    app.run_server(debug=True, port=5004)

Lakeview Center
['Emergency Room', 'Clinic Referral', 'Physician Referral']
None None
         Admit Source Admit Type        Appt Start Time  Care Score  \
4590   Emergency Room  Emergency  2014-01-22 2:19:17 AM           5   
4591   Emergency Room  Emergency  2014-01-22 2:35:16 AM           3   
4592   Emergency Room  Emergency  2014-01-22 3:40:21 AM           3   
11731  Emergency Room  Emergency  2014-02-19 1:59:12 AM           5   
11732  Emergency Room  Emergency  2014-02-19 3:24:36 AM           4   
19565  Emergency Room  Emergency  2014-03-26 1:31:47 AM           5   
19566  Emergency Room  Emergency  2014-03-26 2:06:38 AM           4   
30820  Emergency Room  Emergency  2014-05-14 1:21:36 AM           4   
30821  Emergency Room  Emergency  2014-05-14 2:14:52 AM           2   
30822  Emergency Room  Emergency  2014-05-14 2:35:55 AM           5   
30823  Emergency Room  Emergency  2014-05-14 3:24:35 AM           2   
33241  Emergency Room  Emergency  2014-05-21 1:46:26 AM       

# Caso de Uso: 

Una vez desarrollado el dashboard, se realizará un monitoreo continuo de la clínica **Madison Center** durante el periodo comprendido entre el 01/02/2014 y el 01/20/2014. Este análisis se centrará en las atenciones médicas recibidas en las especialidades de **Emergency Room, Clinic Referral y Physical Referral.**

Como ejemplo representativo, se seleccionaron los valores de Sábado a las 07:00 am para observar el comportamiento de los pacientes en un día y hora específicos. En este intervalo, la clínica brindó atención a 34 pacientes.

El dashboard no solo incluye un mapa de calor que permite visualizar la distribución temporal y espacial de las atenciones, sino también gráficos de tipo boxplot, los cuales facilitan la identificación del tiempo de espera y la calidad de la atención recibida por los pacientes.

Para el grupo de los 34 pacientes mencionados, el tiempo promedio de espera en las especialidades de Emergency y Urgent osciló entre 60 y 80 minutos, mientras que en Elective fue de aproximadamente 30 minutos. En cuanto a la calificación de la atención (care score), las especialidades de Emergency y Urgent obtuvieron una media de 3 puntos, mientras que en Elective se alcanzó una calificación máxima de 4 puntos.

# Conclusiones

Las diversas herramientas de graficación son muy útiles, como en este caso el filtrado y visualización de datos de forma interactiva da una mejor experiencia al usuario para darle a entender lo que busca en el conjunto de datos. 
En este caso plotly con dash muy buenos, rápidos y fáciles de comprender por su estructura de html con los callbacks que funciona como en javascript, proporciona una retroalimentación casi inmediata; depende de la programación de las funciones o su comportamiento pero, es muy bueno. 